# Dataset Builder

Takes raw data and makes a real dataset ready for preprocessing.

## Importing

In [1]:
import pandas as pd

In [24]:
# Get Dataset into pandas dataframe
df_pesagens = pd.read_csv('../data/raw/pesagens2016.csv', sep=';', index_col='PES_ID')
df_roteirizacao = pd.read_csv('../data/raw/rotas.csv', sep=';', index_col='ROTA_ID')

## Info about the raw data

In [25]:
df_pesagens.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119205 entries, 2618746 to 923250
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   LOCDESCARREGO_ID    119205 non-null  int64 
 1   LOCDESCARREGO_DESC  119205 non-null  object
 2   EMP_ID              119205 non-null  int64 
 3   EMP_NOME            119205 non-null  object
 4   ROTA_ID             119205 non-null  object
 5   TPVEICULO_DESC      119205 non-null  object
 6   PES_DATAINI         119205 non-null  object
 7   PES_HRINI           119199 non-null  object
 8   PES_DATAFIM         119205 non-null  object
 9   PES_HRFIM           119202 non-null  object
 10  PES_PESOFIM         119205 non-null  int64 
 11  PES_PESOUTIL        119205 non-null  int64 
 12  COLETA_ID           119205 non-null  int64 
 13  COLETA_DES          119205 non-null  object
 14  ESPECCOLETA_ID      119205 non-null  int64 
 15  ESPECCOLETA_DESC    119205 non-null  object
dtypes

In [26]:
df_roteirizacao.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1228 entries, 01001 to V62
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PERCUSSO_I       1227 non-null   float64
 1   EMP_ID           1228 non-null   int64  
 2   LOCAL_ID         1228 non-null   int64  
 3   ROTA_DESC        1228 non-null   object 
 4   TPCIRCUITO_DESC  1228 non-null   object 
 5   LOCAL_NOME       1228 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 67.2+ KB


In [27]:
df_pesagens.head()

,LOCDESCARREGO_ID,LOCDESCARREGO_DESC,EMP_ID,EMP_NOME,ROTA_ID,TPVEICULO_DESC,PES_DATAINI,PES_HRINI,PES_DATAFIM,PES_HRFIM,PES_PESOFIM,PES_PESOUTIL,COLETA_ID,COLETA_DES,ESPECCOLETA_ID,ESPECCOLETA_DESC
PES_ID,,,,,,,,,,,,,,,,
2618746,7,CTR-CANDEIAS,1,EMLURB,17D,CAMINHAO BAU,2016-01-04,12-30-1899 00:00:00,2016-01-04,12-30-1899 00:00:00,6970,460,2,COLETA DE ENTULHOS,12,ENTULHOS
2619171,7,CTR-CANDEIAS,1,EMLURB,CS02,CAMINHAO BAU,2016-01-05,12-30-1899 00:00:00,2016-01-05,12-30-1899 00:00:00,6610,950,2,COLETA DE ENTULHOS,12,ENTULHOS
2619630,7,CTR-CANDEIAS,1,EMLURB,MN03,CAMINHAO BAU,2016-01-06,12-30-1899 00:00:00,2016-01-06,12-30-1899 00:00:00,6970,1060,2,COLETA DE ENTULHOS,12,ENTULHOS
2618585,7,CTR-CANDEIAS,1,EMLURB,CM01,CAMINHAO CACAMBA,2016-01-03,12-30-1899 00:00:00,2016-01-03,12-30-1899 00:00:00,6850,860,2,COLETA DE ENTULHOS,12,ENTULHOS
2618074,7,CTR-CANDEIAS,1,EMLURB,17D,MUNCK,2016-01-01,12-30-1899 00:00:00,2016-01-01,12-30-1899 00:00:00,11310,920,2,COLETA DE ENTULHOS,12,ENTULHOS


In [28]:
df_roteirizacao.head()

,PERCUSSO_I,EMP_ID,LOCAL_ID,ROTA_DESC,TPCIRCUITO_DESC,LOCAL_NOME
ROTA_ID,,,,,,
01001,1873.0,27,671,COLETA DOM COMPACTADORES,-REA,ZUMBI
01001,1866.0,27,680,COLETA DOM COMPACTADORES,-REA,CORDEIRO
01001,1867.0,27,698,COLETA DOM COMPACTADORES,-REA,IPUTINGA
01002,1697.0,27,744,COLETA DOM COMPACTADORES,-REA,CAXANGA
01003,1835.0,27,728,COLETA DOM COMPACTADORES,-REA,CIDADE UNIVERSITARIA


By looking at both data, as well as looking at the [documentation of the data](http://dados.recife.pe.gov.br/dataset/pesagem-de-coletas-de-residuos), we know that the point of connection for both data is `ROTA_ID` attribute. 

We've gotta be sure that all instances from `pesagens` dataset has a valid `ROTA_ID` that is present on `roteirizacao` dataset, for this, we'll aggregate all valid data and non-valid data will be discarded.

## Aggregating Roteirização Data into Pesagens Dataset